In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('SGD_TESTING.csv')
print(df)
dic = {}

for fr, to, time, type, user_id in zip(df["from"], df["to"], df["timestamp"], df["type"], df["user_id"]):
    if user_id in dic:
        dic[user_id].append([fr, to, type])
    else:
        dic[user_id] = [[fr, to, type]]


hashes = []
for x in range(1,len(dic)):
    dic[x] = np.array(dic[x]).flatten()
    hashes.append(dic[x])


print(len(dic))




        from    to                timestamp type  user_id
0        NaN    S0  2024-05-26 14:07:50.000  Req        1
1         S0    S1  2024-05-26 14:07:50.000  Req        1
2         S1  S1_2  2024-05-26 14:07:50.000  Req        1
3       S1_2    S1  2024-05-26 14:07:50.003  Res        1
4         S1    S0  2024-05-26 14:07:50.003  Res        1
...      ...   ...                      ...  ...      ...
206799    S0    S4  2024-06-05 23:15:31.024  Req     7940
206800    S4  S4_5  2024-06-05 23:15:31.024  Req     7940
206801  S4_5    S4  2024-06-05 23:15:31.034  Res     7940
206802    S4    S0  2024-06-05 23:15:31.034  Res     7940
206803    S0   NaN  2024-06-05 23:15:31.034  Res     7940

[206804 rows x 5 columns]
[[nan, 'S0', 'Req'], ['S0', 'S1', 'Req'], ['S1', 'S1_2', 'Req'], ['S1_2', 'S1', 'Res'], ['S1', 'S0', 'Res'], ['S0', 'S2', 'Req'], ['S2', 'S2_3', 'Req'], ['S2_3', 'S2', 'Res'], ['S2', 'S0', 'Res'], ['S0', 'S3', 'Req'], ['S3', 'S3_1', 'Req'], ['S3_1', 'S3', 'Res'], ['S3', 'S3_2'

In [14]:
from random import shuffle

def jaccard_similarity(set1, set2):
    # intersection of two sets
    intersection = len(np.intersect1d(set1, set2))
    # Unions of two sets
    union = len(np.union1d(set1, set2))
     
    return intersection / union

def create_hash_func(size: int):
    # function for creating the hash vector/function
    hash_ex = list(range(1, len(vocab)+1))
    shuffle(hash_ex)
    return hash_ex

def build_minhash_func(vocab_size: int, nbits: int):
    # function for building multiple minhash vectors
    hashes = []
    for _ in range(nbits):
        hashes.append(create_hash_func(vocab_size))
    return hashes

def create_hash(vector: list):
    # use this function for creating our signatures (eg the matching)
    signature = []
    for func in minhash_func:
        for i in range(1, len(vocab)+1):
            idx = func.index(i)
            signature_val = vector[idx]
            if signature_val == 1:
                signature.append(idx)
                break
    return signature

def shingle(text, k):
    shingle_set = []
    for i in range(len(text)-k +1):
        shingle_set.append(text[i:i+k])
    return set(shingle_set)



{'4S', 'S0', '1S', 'nS', '_1', '_2', '3R', 'S2', '0R', '1_', 'S4', '0n', 'qS', '3S', '_3', '2R', '0S', 'an', '4_', '2_', 'sS', '4R', 'eq', 'nR', 'S3', 'na', '2S', '1R', 'es', '3_', 'S1', 'Re'}
{'4S', 'S0', '1S', 'nS', '_1', '_2', '_5', '3R', 'S2', '0R', '1_', 'S4', '0n', 'qS', '3S', '_3', '2R', '0S', '5R', 'an', '4_', '5S', '2_', 'sS', '4R', 'eq', '_4', 'nR', 'S3', 'na', '2S', '1R', 'es', '3_', 'S1', 'Re'}
['Req' 'Res' 'S0' 'S1' 'S1_2' 'S2' 'S2_1' 'S2_2' 'S2_3' 'S2_4' 'S2_5' 'S3'
 'S3_1' 'S3_2' 'S3_3' 'S4' 'S4_2' 'S4_5' 'nan']
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0] [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]


In [20]:
#shingle
arr1 = shingle("".join(dic[1]), 2)
arr2 = shingle("".join(dic[2]), 2)
print(arr1)
print(arr2)

#vocab for 1hot
vocab = np.union1d(dic[1], dic[2])
print(vocab)

#1hot-encoding
x = [1 if item in vocab else 0 for item in arr1]
y = [1 if item in vocab else 0 for item in arr2]

# we create 20 minhash vectors
minhash_func = build_minhash_func(len(vocab), 20)

arr1_sig = create_hash(x)
arr2_sig = create_hash(y)

print(arr1_sig)
print(arr2_sig)

print(jaccard_similarity(arr1_sig, arr2_sig))


[1, 7, 10, 7, 7, 10, 10, 10, 1, 1, 7, 1, 10, 10, 1, 10, 10, 7, 10, 1]
[8, 8, 8, 8, 11, 1, 11, 11, 11, 1, 8, 1, 1, 1, 1, 8, 1, 8, 11, 11]
0.2
